In [1]:
from libra.retrievers import VectorRetriever
from libra.vectordb import MilvusStorage, QdrantStorage
from libra.embeddings import SentenceTransformerEncoder, OpenAIEmbedding
from qdrant_client import QdrantClient

import os
from dotenv import load_dotenv

load_dotenv(override=True)

embedding_instance = SentenceTransformerEncoder()

/home/gcp_dpf/libra-core/.venv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
client = QdrantClient(
    path='../../libra_qdrant.db'
)

job_title_storage_instance = QdrantStorage(
    client=client,
    vector_dim=embedding_instance.get_output_dim(),
    collection_name="MBInfo"
)
job_title_storage_instance.clear()

In [3]:
mb_info_retriever = VectorRetriever(
    embedding_model=embedding_instance, 
    storage=job_title_storage_instance
)

### Adding Company Profile

In [4]:
from libra.loaders import read_file
from io import BytesIO
import json
from libra.utils import json_to_well_formatted

company_profiles = []


file_path = os.path.join('..', '..', 'data', 'mb_company_profile_summarized.csv')
with open(file_path, "rb") as f:
    file = BytesIO(f.read())
    file.name = "mb_company_profile_summarized.csv"
    json_file = read_file(file)
    for doc in json_file.docs:
        doc_content = json.loads(doc['page_content'])    
        company_profiles.append({
            "embed": doc_content['summary'],
            "content": f"""
topic: {doc_content['topic']},
summary: {doc_content['summary']}
content: {doc_content['content']}
            """
        })

In [5]:
mb_info_retriever.process(data=company_profiles, batch_size=1)

100%|██████████| 128/128 [00:35<00:00,  3.63it/s]


In [ ]:
results = mb_info_retriever.query(query="MB có bao nhiêu khách hàng?", top_k=4)

for entry in results:
    print(entry['text'])

### Adding jobs

In [11]:
job_storage_instance = QdrantStorage(
    client = client,
    vector_dim=embedding_instance.get_output_dim(),
    collection_name="jobs"
)
job_storage_instance.clear()

In [12]:
job_retriever = VectorRetriever(
    embedding_model=embedding_instance,
    storage=job_storage_instance
)

In [13]:
import re

def parse_salaries(salary_data):
    salaries = ("", "")
    lines = salary_data.split('\n')
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        
        # Check for "a - b VND" format
        match = re.match(r'(\d{1,3}(?:,\d{3})*) - (\d{1,3}(?:,\d{3})*)\s*VNĐ', line)
        if match:
            min_salary = match.group(1).replace(',', '')
            max_salary = match.group(2).replace(',', '')
        else:
            # For other formats, set min and max to the same value
            # Remove "VNĐ" and any leading/trailing whitespace
            value = line.replace('VNĐ', '').strip()
            
            if value in ['Lương thỏa thuận', 'Cạnh Tranh']:
                min_salary = max_salary = value
            else:
                # Try to extract a number if present
                number_match = re.search(r'\d{1,3}(?:,\d{3})*', value)
                if number_match:
                    salary_value = int(number_match.group().replace(',', ''))
                    min_salary = max_salary = salary_value
                else:
                    min_salary = max_salary = value

        salaries = (min_salary, max_salary)
    
    return salaries

def is_number(s):
    return s.isdigit()

In [14]:
from libra.loaders import read_file
from io import BytesIO
import json
from pprint import pprint

jobs = []

keys_to_extract = ["title", "workplace", "industry", "welfare", "min_salary", "max_salary"]

file_path = os.path.join('..', '..', 'data', 'jobs_data.json')
with open(file_path, "rb") as f:
    file = BytesIO(f.read())
    file.name = "jobs_data.json"
    json_file = read_file(file)
    for doc in json_file.docs:
        doc_content = json.loads(doc['page_content']) 
        a, b = parse_salaries(doc_content['salary'])
        if is_number(a):
            a +=' VNĐ' # type: ignore
            
        if is_number(b):
            b +=' VNĐ' # type: ignore
        
        doc_content['min_salary'] = a
        doc_content['max_salary'] = b
            
        job = {
            "embed": str({key: doc_content[key] for key in keys_to_extract}),
            "content": str(doc_content)
        }
        jobs.append(job)

In [15]:
jobs[-1]['content']

"{'_id': {'$oid': '66c015512d62a130f80804b2'}, 'url': 'https://tuyendung.mbbank.com.vn/viec-lam/chuyen-vien-cvcc-quan-tri-rui-ro-mo-hinh-khoi-quan-tri-rui-ro.35aa58be.html', 'title': 'Chuyên viên/CVCC Quản trị rủi ro mô hình - Khối Quản trị rủi ro', 'workplace': 'Hà Nội', 'job_rank': 'Nhân viên', 'job_form': 'Nhân viên chính thức', 'experience': '1 - 10 Năm', 'salary': 'Lương thỏa thuận', 'industry': 'Ngân hàng, Tài chính / Đầu tư', 'deadline_applications': '30/05/2024', 'welfare': ['Bảo hiểm', 'Du Lịch', 'Thưởng', 'Chăm sóc sức khỏe', 'Đào tạo', 'Tăng lương'], 'job_detail': [], 'job_requirements': [], 'job_tags': ['Chuyên viên/CVCC Quản trị rủi ro mô hình - Khối Quản trị rủi ro'], 'min_salary': 'Lương thỏa thuận', 'max_salary': 'Lương thỏa thuận'}"

In [15]:
job_storage_instance.clear()

In [16]:
job_retriever.process(jobs, batch_size=1)

100%|██████████| 365/365 [03:44<00:00,  1.63it/s]


In [17]:
for entry in job_retriever.query("nghề UB ở Hải Phòng", top_k=5):
    print(entry['text'])

{'_id': {'$oid': '66c015512d62a130f8080447'}, 'url': 'https://tuyendung.mbbank.com.vn/viec-lam/chuyen-vien-ub-chuyen-vien-tu-van-cn-nam-hai-phong.35aa78b2.html', 'title': 'Chuyên viên UB (Chuyên viên Tư vấn) - CN. Nam Hải Phòng', 'workplace': 'Hải Phòng', 'job_rank': 'Nhân viên', 'job_form': 'Nhân viên chính thức', 'degree': 'Đại học', 'experience': '1 Năm', 'salary': '8,000,000 - 15,000,000  VNĐ', 'industry': 'Dịch vụ khách hàng, Ngân hàng, Tài chính / Đầu tư', 'deadline_applications': '30/05/2024', 'welfare': ['Bảo hiểm', 'Du Lịch', 'Phụ cấp', 'Du lịch nước ngoài', 'Đồng phục', 'Thưởng', 'Chăm sóc sức khỏe', 'Đào tạo', 'Tăng lương', 'Công tác phí', 'Phụ cấp thâm niên', 'Chế độ nghỉ phép', 'Chế độ tập thể dục'], 'job_detail': ['Thực hiện công tác bán hàng tại sàn giao dịch chi nhánh: tư vấn và bán các sản phẩm, dịch vụ của MB cho Khách hàng cá nhân bao gồm: Huy động vốn, tài khoản thanh toán, thẻ, ngân hàng điện tử…\xa0Bán chéo sản phẩm dịch vụ của MB…', 'Thực hiện cung cấp dịch vụ tạ

### Adding jobs title

In [2]:
from qdrant_client import QdrantClient

client = QdrantClient(
    path='../../libra_qdrant.db'
)

job_title_storage_instance = QdrantStorage(
    client=client,
    vector_dim=embedding_instance.get_output_dim(),
    collection_name="job_title"
)
job_title_storage_instance.clear()

In [3]:
job_title_values = [
    'Chuyên viên Khách hàng Cá nhân',
    'Chuyên viên Tập sự Khách hàng Cá nhân',
    'Chuyên viên Tập sự Khách hàng Doanh nghiệp',
    'Chuyên viên Khách hàng Doanh nghiệp',
    'Chuyên gia Phát triển dịch vụ (API)',
    'Chuyên viên Phát triển dịch vụ thẻ Khách hàng doanh nghiệp (BA thẻ)',
    'Cộng tác viên Tuyển dụng', 'Chuyên viên UB',
    'Chuyên viên Vận hành hệ thống', 'Data Platform Engineer',
    'Quality Assurance (QA)', 'UI UX Designer',
    'Chuyên viên Thúc đẩy bán Sản phẩm Tín dụng', 'Giao dịch viên',
    'Kiểm soát viên', 'Kiểm toán viên (mảng Mô hình, Dữ liệu)',
    'Phó phòng Khách hàng Cá nhân', 'Trưởng phòng Khách hàng Cá nhân',
    'Chuyên viên và Chuyên viên cao cấp Quản trị rủi ro mô hình',
    'Chuyên viên và Chuyên viên cao cấp Kiểm định Mô hình rủi ro',
    'Chuyên viên cao cấp Phân tích nghiệp vụ',
    'Chuyên viên và Chuyên viên cao cấp Xây dựng Mô hình rủi ro',
    'Red Team', 'Pentester', 'Chuyên viên kiến trúc an toàn thông tin',
    'Chuyên viên Giải pháp nền tảng',
    'Chuyên viên Phát triển nguồn lực',
    'Trưởng phòng Khách hàng Doanh nghiệp',
    'Nhân viên hành chính (NS)', 'Kiểm ngân', 'Giám đốc Dịch vụ',
    'Chuyên viên Tài trợ thương mại',
    'Chuyên viên Phát triển Kinh doanh Bancassurance',
    'Chuyên viên và Chuyên viên cao cấp Quản trị rủi ro tích hợp',
    'Chuyên viên Hỗ trợ', 'Giám đốc Quan hệ Khách hàng',
    'Chuyên viên Dịch vụ Tài trợ Thương mại',
    'Chuyên viên Kiểm thử tự động (Automation Tester)',
    'Chuyên viên Khách hàng lớn', 'Network Administrator',
    'Chuyên viên và Chuyên viên cao cấp Phát triển thương hiệu',
    'Chuyên viên và Chuyên viên cao cấp Quản trị nợ',
    'Chuyên viên Quản lý ứng dụng', 'AI Engineer', 'Kỹ sư Devsecops',
    'Chuyên viên cao cấp và Chuyên gia Tư vấn pháp chế',
    'Phó Giám đốc Chi nhánh (SME)', 'Chuyên viên Tập sự UB',
    'Chuyên viên Khách hàng Cá nhân (aRM)',
    'Chuyên viên Cao cấp và Chuyên gia Chính sách và chất lượng dữ liệu',
    'Data Architect', 'Data Engineer', 'Business Analyst',
    'Chuyên viên Cao cấp và Chuyên gia Quản lý cấu trúc dữ liệu',
    'Tester',
    'Chuyên viên và Chuyên viên Cao cấp Mô hình kinh doanh (Data scientist)',
    'Chuyên viên và Chuyên viên Cao cấp Công nghệ mô hình (AI/ML Ops)',
    'Chuyên viên và Chuyên viên Cao cấp Kiểm định mô hình (Model Validation)',
    'Chuyên viên/CVCC Phân tích Khách hàng cá nhân',
    'Chuyên viên và Chuyên viên Cao cấp Phân tích Khách hàng doanh nghiệp',
    'Data Scientist',
    'Chuyên viên và Chuyên viên Cao cấp Phân tích quản trị',
    'Data Analyst',
    'Chuyên viên Phát triển kinh doanh SME Factory Micro Lending Squad',
    'Chuyên viên kinh doanh thẻ',
    'DevOps Engineer (Python, Nodejs, Java)', 'Lập trình viên Backend',
    'Lập trình viên Mobile',
    'Chuyên viên và Chuyên viên cao cấp Giám sát rủi ro tín dụng',
    'Chuyên viên và Chuyên viên cao cấp Chính sách rủi ro tín dụng',
    'Chuyên Viên Quản lý tuân thủ', 'Chuyên viên Điều tra',
    'Chuyên viên và Chuyên viên cao cấp Quản lý pháp chế hệ thống',
    'Kỹ Sư Phát Triển BackEnd', 'Kỹ sư Phát triển Fullstack',
    'Kỹ sư Phát triển AI', 'Kỹ sư phát triển Mobile',
    'Chuyên viên Quản lý xây dựng', 'Title: Giám đốc Dịch vụ',
    'Giám đốc Phòng giao dịch', 'Giám đốc UB',
    'Chuyên viên và Chuyên viên cao cấp Đầu tư chiến lược và M&A',
    'Chuyên viên/ Chuyên viên cao cấp Quản trị rủi ro thị trường'
]

embed_jobs_title = [
    {"embed": job_title, "content": job_title} for job_title in job_title_values
]

In [7]:
job_title_retriever = VectorRetriever(
    embedding_model=embedding_instance,
    storage=job_title_storage_instance,
    similarity_threshold=0.6
)

In [5]:
job_title_retriever.process(embed_jobs_title, 1)

100%|██████████| 83/83 [00:13<00:00,  6.22it/s]


In [11]:
len(job_title_retriever.query("Lập trình viên", top_k=50))

20